### Bibliotecas

In [1]:
from urllib.request import Request, urlopen, urlretrieve
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
import pandas as pd
import openpyxl
import copy
from functools import reduce
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import re
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


### 1. Extração dos links das notícias

In [2]:
#Tribunal
institution = input('digite a sigla do tribunal da Justiça Eleitoral:').lower().strip()
institution_pattern = re.compile('(tse)|(tre-(ac|al|ap|am|ba|ce|df|es|go|ma|mt|ms|mg|pa|pb|pr|pe|pi|rj|rn|rs|ro|rr|sc|sp|se|to)$)')
match = institution_pattern.match(institution)

while not match:
    print('A sigla não é válida. Digite novamente.')
    institution = input('digite a sigla do tribunal da Justiça Eleitoral:').lower().strip()
    institution_pattern = re.compile('(tse)|(tre-(ac|al|ap|am|ba|ce|df|es|go|ma|mt|ms|mg|pa|pb|pr|pe|pi|rj|rn|rs|ro|rr|sc|sp|se|to)$)')
    match = institution_pattern.match(institution)

# Mês e ano
year = input('Digite o ano:').strip()
year_pattern = re.compile('[2][0][2][123]')
match = year_pattern.match(year)

while not match:
    print('O ano não é válido. Digite novamente.')
    year = input('digite o ano:').strip()
    year_pattern = re.compile('[2][0][2][123]')
    match = year_pattern.match(year)
    

    

month = input('digite o mês ou pressione enter para seguir adiante:').lower().strip()
months = ['', 'janeiro', 'fevereiro', 'março', 'abril', 
          'maio', 'junho', 'julho', 'agosto', 
          'setembro', 'outubro', 'novembro', 'dezembro']
          
while month not in months:
    print('O mês não é válido. Digite novamente')
    month = input('digite o mês:').lower().strip()
    
if month == '':
    month = 'mês'
    

digite a sigla do tribunal da Justiça Eleitoral:tre-sp
Digite o ano:2023
digite o mês ou pressione enter para seguir adiante:


In [3]:
#driver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)


In [4]:
url = f'https://www.{institution}.jus.br/comunicacao/noticias'
driver.get(url)
if institution == 'tse':
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="modal-lgpd"]/div/div/div[2]/button'))).click()
else:
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="modal-lgpd"]/div/div/div[3]/button'))).click()
driver.find_element(By.ID, 'ano-noticias').send_keys(year)
driver.find_element(By.ID, 'mes-noticias').send_keys(month)
driver.find_element(By.XPATH, '//*[@id="form-navagacao-noticias"]/fieldset/button').click()

#iterador para rolar a página de notícias 
for i in range(1000):
    try:
        driver.find_element(By.XPATH, '//*[@id="news-list-read-more-button"]').click()
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="news-list-read-more-button"]'))).click()
    except:
        pass

#lista de notícias 
news = driver.find_elements(By.CLASS_NAME, 'news-list-title')

# lista de links das noticias
links = [n.find_element(By.TAG_NAME, 'a').get_attribute('href') for n in news]



In [5]:
# dicionário para criar dataframe com os links (para rodar em máquinas sem autorização para usar o Selenium)
df_links = pd.DataFrame({'INDEX': range(len(links)),
                         'LINK': links})

# renomeando o 'mês':
if month == 'mês':
    month = ''
else:
    month = '_' + month

# exportando planilha de links
df_links.to_excel(f'links_{institution}{month}_{year}.xlsx', index=False)

    

### 3. Acessar links e extrair informações

In [6]:
list_dict_links = []

for l in df_links['LINK']:

    response = urlopen(l)
    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')

    dict_link = {}
    
    #data
    dict_link['DATA'] = soup.find('div', {"class": "data-hora"}).get_text().split()[0]
    
    #hora
    dict_link['HORA'] = soup.find('div', {"class": "data-hora"}).get_text().split()[1]

    #título
    title = soup.find('h2', {"class": "noticia-titulo"})
    dict_link['TÍTULO DA MATÉRIA'] = soup.find('h2', {"class": "noticia-titulo"}).get_text()

    #subtítulo
    dict_link['SUBTÍTULO'] = title.findNext().get_text()

    #assunto/tema
    dict_link['ASSUNTO/TEMA'] = None

    #tags
    tags = []
    tags_name = soup.findAll('li', {"class": "tag"})
    for i in tags_name:
        tag_text = i.findNext().get('title').replace('Link: ', '')
        tags.append(tag_text)

    tags_list = (tags)
    tc = reduce(lambda x, y: x + ' , ' + y, tags_list)

    dict_link['TAGS'] = tc

    #tempo de leitura:
    time = soup.find('div', {"class": "tempo-leitura"}).get_text().split()

    for i in time:
        try:
            time.remove('de')
            time.remove('leitura')
        except:
            pass


    #tempo de leitura: removendo caracteres e convertendo para número

    chars = ['m','s']

    chars = set(chars)

    list_time = []

    for t in time:
        if t[2] == 'm':
            m = int(''.join(c for c in t if c not in chars))
            minutes = m * 60
            list_time.append(minutes)
        if t[2] == 's':
            seg = int(''.join(c for c in t if c not in chars))
            list_time.append(seg)

    reading_time = sum(list_time)

    dict_link['TEMPO DE LEITURA (em segundos)'] = reading_time   

    #link
    dict_link['LINK'] = l

    #conteúdo das notícias
    dict_link['CONTEÚDO'] = soup.find('div', {"class": "tab-pane active"}).get_text()
    
    #adicionando dicionário na lista de notícias
    list_dict_links.append(dict_link)
    
### 4. Gerar planilha com as notícias

if month == 'mês':
    month = ''
else:
    month = '_' + month

date = year + month

df = pd.DataFrame(list_dict_links)
df.to_excel(f'noticias_{institution}_{month}_{year}.xlsx', index=False)